<a href="https://colab.research.google.com/github/pavi-ninjaac/NuralNetwork_scratch/blob/main/CNN/Forword/ConvolutionalLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# conolutional layer
import numpy as np

In [18]:
class con2D:
  def __init__(self, filter , kernal_size, img_channel,strides = (1,1) ):
    self.filter = None
    self.bias = None
    """Convolute the image with the filter and strids"""
    
    
    f_channel = img_channel               #filter channel must match with the image channel
    self.no_filters = filter                   # number of filters
    f_dim , _ = kernal_size
    self.f_dim =  f_dim              # get the filter size
    self.s , _ = strides
    f_size = (self.no_filters , f_channel , f_dim , f_dim) # get the strides dimentions

    #initialize the filter
    """ Initalize the filter with the noraml distribution with the standard deviation inversely proportional the square root of the number of units"""
    sdv = 1.0 / np.sqrt(np.prod(f_size))
    #self.filter = np.random.normal(loc = 0, scale = sdv, size = f_size)
    self.filter = np.array([[[1,0,1] , [2,1,1] , [0,1,2]]])
    self.bias = np.zeros((self.no_filters , 1)) # each for the one filters    

  
  #forward propogaton
  def forward_propogation(self, image):
    print(image.shape)
    (img_channel , input_dim , _ ) = image.shape #input dimensions channel first dimention 
    
    #calculate the output dimension, so according to that i can create the empty array to store value
    out_dim = int(((input_dim - self.f_dim) / self.s) + 1)
    print('out put dimention',out_dim)
    #create the empty array with output dimetions
    self.output = np.zeros((1 , out_dim, out_dim )) #syntax np.zeros(channel , row , col)
    for curr_filter in range(self.no_filters): # for loop over the number of filters
      curr_y = out_y =0
      
      #move vertically over the image
      while (curr_y + self.f_dim) <= input_dim:
        curr_x = out_x = 0
        #move horizondally
        while (curr_x + self.f_dim) <= input_dim:
          
          """perform convolution operation on the image"""
          self.output[curr_filter , curr_y , curr_x] = np.sum(self.filter[curr_filter] * image[: , curr_y : curr_y + self.f_dim , curr_x : curr_x + self.f_dim]) #+ self.bias[curr_filter]

          curr_x += self.s
          out_x +=1
        curr_y += self.s
        out_y += 1
    print(self.output)
    return self.output

    def back_propogation(self):
      pass



In [19]:
class MaxPooling2D(con2D):
  def __init__(self , pool_size = 1, strides=1):
    self.pool_size = pool_size #getting the kernal size for pooling the layer out
    self.strides = strides
    
  def forward_propogation(self , image):

    """ downsample of the image using the kernal size and strides"""
    img_channel , h_prev , w_prev  = image.shape    # shape of the image
    # calculate output dimensions after the maxpooling operation.
    h = int((h_prev - self.pool_size)/self.strides)+1 
    w = int((w_prev - self.pool_size)/self.strides)+1
    
    # create a matrix to hold the values of the maxpooling operation.
    self.output = np.zeros((img_channel, h, w))
    # slide the window over every part of the image using stride s. Take the maximum value at each step.
    for i in range(img_channel):
        curr_y = out_y = 0
        # slide the max pooling window vertically across the image
        while curr_y + self.pool_size <= h_prev:
            curr_x = out_x = 0
            # slide the max pooling window horizontally across the image
            while curr_x + self.pool_size <= w_prev:
                # choose the maximum value within the window at each step and store it to the output matrix
                self.output[i, out_y, out_x] = np.max(image[i, curr_y:curr_y+self.pool_size, curr_x:curr_x+self.pool_size])
                curr_x += self.strides
                out_x += 1
            curr_y += self.strides
            out_y += 1
    print(self.output)
    return self.output

    def back_propogation(self):
      pass


In [32]:
class Flatten(con2D):
  """ get the strignt single channel fully connet layer """
  def __init__(self):
    self.flatten_image = None
    
  def forward_propogation(self , image):
    image_channel , input_dim , _ = image.shape
    self.flatten_image = image.flatten()
    print(self.flatten_image)
    return self.flatten_image 
  
  def get_output_size(self):
    return self.flatten_image.shape[0]

  def back_propogation(self):
      pass

In [33]:
class FClayer(Flatten):
  def __init__(self , input_size , output_size):
    self.input = None
    self.input_size = input_size
    self.output = None
    # initialize the weights using the normal distribution
    self.weights = np.random.rand(self.input_size , output_size) - 0.5
    self.bias =  np.random.rand(1,output_size) - 0.5 
  
  def forward_propogation(self , input):
    self.input = input
    self.output = np.dot(self.input , self.weights) + self.bias
    print(self.output)
    return self.output

In [22]:
def softmax_funtion(output):
  """ pass the raw predicted data to the activation function to get the probability"""
  out = np.exp(output) # exponent the predicted output
  return out / np.sum(out)

In [23]:
def catogarical_cross_entropy(prob , label):
  """ calculate the categorical cross entropy loss"""
  return -np.sum(label * np.log(prob)) #multiply the output with the log of probability

In [52]:
class softmax(FClayer):
  def __init__(self, actvation = softmax_funtion , activation_derivated = None): #default parameter setting
    self.actvation = actvation #show the activation from the outside class
    self.activation_derivated = activation_derivated

  def forward_propogation(self , input_data):
    self.input = input_data 
    self.output = self.actvation(self.input)
    print('soft max activation',self.output)
    return self.output

  def back_propogation(self ,  output_error , learning_rate):
    return self.activation_derivated(self.input) * output_error

In [25]:
class Relu(con2D):
  """ pass through the relu activation function"""
  def __init__(self):
    self.output = None

  def forward_propogation(self,input):
    input[input<=0] = 0
    self.output = input
    print(self.output)
    return self.output
  
  def back_propogation(self):
    pass


In [56]:
class network:
  def __init__(self):
    self.layers = []

  #add the layer in the model
  def add(self , layer):
    self.layers.append(layer)
  #set the loss function to use it out
  def setloss(self, loss , loss_derivated):
    self.loss = loss
    self.loss_derivated = loss_derivated

  def fit(self , x_train ,  y_train , epoches):
    sampels = len(x_train)
    #loop over for epoches
    for i in range(epoches):
      err = 0
      for j in range(sampels):
        print('sample' , j)
        output = x_train[j] 
        for layer in self.layers:
          print(layer)
          output = layer.forward_propogation(output)

        #compute the loss
        err += self.loss(y_train[j] , output)
        print('loss ',err)
        #backpropogation
        #error = self.loss_derivated(y_train[j] , output)
        #print(error) 
    print(output)



In [59]:
#build the model 
model = network()
model.add(con2D(filter = 1 , kernal_size =(3,3) , strides = (1,1) , img_channel =1))
model.add(Relu())
model.add(MaxPooling2D(pool_size = 2))
model.add(Relu())
model.add(Flatten())
model.add(FClayer(4,3))
model.add(FClayer(3,2))
model.add(softmax())

In [60]:
i = np.array([[[[1,2,1,0,1] , [2,1,1,2,1] , [3,2,1,0,0] , [0,1,2,3,1] , [2,1,1,0,2]]]])
print(i.shape)
#set the loss to compute the loss
model.setloss(loss = catogarical_cross_entropy , loss_derivated = None)
model.fit(i ,[1,0] ,1)

(1, 1, 5, 5)
sample 0
(1, 5, 5)
out put dimention 3
[[[12.  8.  7.]
  [17. 16.  9.]
  [10. 10. 13.]]]
[[[12.  8.  7.]
  [17. 16.  9.]
  [10. 10. 13.]]]
[[[17. 16.]
  [17. 16.]]]
[[[17. 16.]
  [17. 16.]]]
[17. 16. 17. 16.]
[[ 7.2314444  -4.13163617  8.80067874]]
[[ 0.61187041 -0.44427832]]
soft max activation [[0.74195388 0.25804612]]
loss  0.0
[[0.74195388 0.25804612]]


In [ ]:
f_size = (2,3,3)
sdv = 1.0 / np.sqrt(np.prod(f_size))
sd = np.random.normal(loc = 0, scale = sdv, size = f_size)
sd

array([[[-0.2358713 ,  0.26419976, -0.05489258],
        [-0.07005125, -0.03529608, -0.13862036],
        [-0.42406881,  0.16785173, -0.26264137]],

       [[ 0.27956213,  0.14691208,  0.31053885],
        [-0.21537503,  0.16352159,  0.0658891 ],
        [-0.65043857,  0.19225158,  0.08999125]]])

In [30]:
s = np.array([[[1,0,1] , [2,1,1] , [0,1,2]]])
s

array([[[1, 0, 1],
        [2, 1, 1],
        [0, 1, 2]]])

In [3]:
i = np.array([ [[1,2,1,0,-1] , [-2,1,1,2,1] , [3,2,1,0,0] , [0,1,2,3,1] , [2,1,1,0,2]]])
i[i<=0] = 0
i

array([[[1, 2, 1, 0, 0],
        [0, 1, 1, 2, 1],
        [3, 2, 1, 0, 0],
        [0, 1, 2, 3, 1],
        [2, 1, 1, 0, 2]]])

In [ ]:
class a:
   
  def __init__(self):
    self.i = 2
  def d(self):
    self.i = 10
    print(self.i)
  def geti(self):
    return self.i
a().d()  
class b(a):
  print(super().geti())

10


RuntimeError: ignored

In [31]:
s.flatten()

array([1, 0, 1, 2, 1, 1, 0, 1, 2])